In [ ]:
#################### DEFINIZIONE DEI 3 LLM ####################

from transformers import DPRContextEncoder, DPRQuestionEncoder
from transformers import AutoTokenizer, AutoModelForSeq2SeqLM

context_tokenizer = AutoTokenizer.from_pretrained('facebook/dpr-ctx_encoder-single-nq-base');
context_encoder = DPRContextEncoder.from_pretrained('facebook/dpr-ctx_encoder-single-nq-base');

query_tokenizer = AutoTokenizer.from_pretrained('facebook/dpr-question_encoder-single-nq-base');
query_encoder = DPRQuestionEncoder.from_pretrained('facebook/dpr-question_encoder-single-nq-base');

generator_tokenizer = AutoTokenizer.from_pretrained("google/flan-t5-base");
generator = AutoModelForSeq2SeqLM.from_pretrained("google/flan-t5-base"); # buono per Q-A con contesto
generator_tokenizer.pad_token = generator_tokenizer.eos_token
generator.config.pad_token_id = generator_tokenizer.pad_token_id

/usr/local/lib/python3.12/dist-packages/huggingface_hub/utils/_auth.py:94: UserWarning: 
The secret `HF_TOKEN` does not exist in your Colab secrets.
To authenticate with the Hugging Face Hub, create a token in your settings tab (https://huggingface.co/settings/tokens), set it as secret in your Google Colab and restart your session.
You will be able to reuse this secret in all of your notebooks.
Please note that authentication is recommended but still optional to access public models or datasets.
  warnings.warn(


tokenizer_config.json:   0%|          | 0.00/28.0 [00:00<?, ?B/s]

config.json:   0%|          | 0.00/492 [00:00<?, ?B/s]

vocab.txt: 0.00B [00:00, ?B/s]

tokenizer.json: 0.00B [00:00, ?B/s]

...

tokenizer.json: 0.00B [00:00, ?B/s]

special_tokens_map.json: 0.00B [00:00, ?B/s]

config.json: 0.00B [00:00, ?B/s]

model.safetensors:   0%|          | 0.00/990M [00:00<?, ?B/s]

generation_config.json:   0%|          | 0.00/147 [00:00<?, ?B/s]

In [ ]:
!pip install -q faiss-cpu

   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 23.8/23.8 MB 21.2 MB/s eta 0:00:00


In [ ]:
#################### SPAZIO VETTORIALE ####################
""" Serve sapere la dimensione dell'embedding. Notare che ogni frase diventa un solo vettore """

tokens_info = context_tokenizer(["ciao", "buongiorno"], return_tensors = 'pt',
                                padding = True, truncation = True, max_length = 256)
context_encoder(**tokens_info)["pooler_output"] # shape: [Bs, 768]

embedding_dim = 768

import numpy as np
import faiss

vector_space = faiss.IndexFlatL2(embedding_dim) # crea uno spazio vettoriale VUOTO a 'embedding_dim' dimensioni
# vector_space.add(....) # per aggiungere vettori allo spazio vettoriale

In [ ]:
!pip install -q wget

  Preparing metadata (setup.py) ... done


In [ ]:
#################### PREPARAZIONE DEL DATASET ####################
import wget

filename = 'companyPolicies.txt'
url = 'https://cf-courses-data.s3.us.cloud-object-storage.appdomain.cloud/6JDbUb_L3egv_eOkouY71A.txt'

# Use wget to download the file
wget.download(url, out = filename)
print('file downloaded')

with open("companyPolicies.txt", 'r', encoding = 'utf-8') as file:
        text = file.read()
print(text[0:400])

file downloaded
1.	Code of Conduct

Our Code of Conduct outlines the fundamental principles and ethical standards that guide every member of our organization. We are committed to maintaining a workplace that is built on integrity, respect, and accountability.
Integrity: We hold ourselves to the highest ethical standards. This means acting honestly and transparently in all our interactions, whether with colleagues


In [ ]:
# splittiamo in paragrafi: questo e' il chunking ma senza overlapping

def chunking(text):
  paragraphs = text.split('\n') # chunking secco
  paragraphs = [paragraph.strip() for paragraph in paragraphs if len(paragraph.strip()) > 0]
  return paragraphs

paragraphs = chunking(text) # e' una lista, corta, che puo' essere fornita al context_encoder
len(paragraphs) # 76

76

In [ ]:
#################### CONTEXT EMBEDDING ####################
""" lo facciamo facile senza definire una batch_size, embeddando tutto direttamente
    perche' il documento e' piccolo. Se il dataset fosse enorme e non stesse nella RAM,
    avrei dovuto usare la strategia sotto 'encode_in_batches' o simili. """

tokens_info = context_tokenizer(paragraphs, return_tensors = 'pt', padding = True, truncation = True, max_length = 256)
embedded_context = context_encoder(**tokens_info)["pooler_output"] # shape: [numero_paragrafi = 76, 768]

import torch

def encode_in_batches(text_list, tokenizer, encoder, batch_size = 32):
  all_embeddings = []
  for i in range(0, len(text_list), batch_size):
    batch_text = text_list[i: i + batch_size]
    tokens = tokenizer(
        batch_text,
        return_tensors = 'pt',
        padding = True,
        truncation = True,
        max_length = 256)

    with torch.no_grad():
      output = encoder(**tokens).pooler_output

    all_embeddings.append(output) # shape: [batch_size: 768]

  return torch.cat(all_embeddings, dim = 0) # shape: [numero_paragrafi, 768]

In [ ]:
#################### AGGIUNGERE IL CONTESTO ALLO SP.VETT. ####################
""" Oltre ad aggiungere il contesto embeddato allo spazio vettoriale, calcoliamo
    anche la distanza massima fra i vettori. Questo ci servira' per normalizzare
    la distanza fra query-context e trovare i vettori piu' vicini alla query """

embedded_context_np = embedded_context.detach().numpy()
vector_space.add(embedded_context_np) # lo sp. vett. va poopolato da tensori numpy
vector_space.ntotal # 76 vettori in totale
vector_space.d # di dimensione 768

768

In [ ]:
from scipy.spatial.distance import pdist, squareform

dist_matrix = squareform(pdist(embedded_context_np, metric = 'euclidean'))

max_distance = np.max(dist_matrix)
max_distance_squared = max_distance ** 2

print(f"Max squared L2 distance: {max_distance_squared}")

Max squared L2 distance: 209.3562521069382


In [ ]:
#################### RETRIEVAL ####################
""" Data una query, vogliamo ritrovare i top-k chunk piu' vicini"""

query = "Which is the Drug and Alcohol Policy?"
tokens_info = query_tokenizer(query, return_tensors = 'pt', padding = True, truncation = True, max_length = 256)
embedded_query = query_encoder(**tokens_info)["pooler_output"]
embedded_query_np = embedded_query.detach().numpy()

In [ ]:
top_k = 5 # numero di chunks da ritornare
distances, indices = vector_space.search(embedded_query_np, k = top_k)

distances # array([[85.04201, 90.60243, 91.88905, 95.73875, 97.32027]])
indices # array([[48, 38, 28, 59,  8]])
normalized_distances = np.round(distances[0]/max_distance_squared * 100,2) # array([40.62, 43.28, 43.89, 45.73, 46.49])

In [ ]:
# tutto in una funzione

def retrieve_from_query(query, top_k, display = False):
  tokens_info = query_tokenizer(query, return_tensors = 'pt', padding = True, truncation = True, max_length = 256)
  embedded_query = query_encoder(**tokens_info)["pooler_output"]
  embedded_query_np = embedded_query.detach().numpy()
  distances, indices = vector_space.search(embedded_query_np, k = top_k)
  normalized_distances = np.round(distances[0]/max_distance_squared * 100,2)
  if display == True:
    print(f"\t\t\t\t\t\t\tTop {top_k} relevant contexts:")
    for i, index in enumerate(indices[0]):
      print("-"*160)
      print(f"Result number {i+1} with (relative) distance {normalized_distances[i]}%: \n {paragraphs[index]}")
  return distances[0], indices[0], normalized_distances

In [ ]:
query = "Which is the Drug and Alcohol Policy of our company?"
distances, indices, normalized_distances = retrieve_from_query(query, 5, display = True)

							Top 5 relevant contexts:
----------------------------------------------------------------------------------------------------------------------------------------------------------------
Result number 1 with (relative) distance 36.96%: 
 Policy Objective: The Drug and Alcohol Policy is established to establish clear expectations and guidelines for the responsible use of drugs and alcohol within the organization. This policy aims to maintain a safe, healthy, and productive workplace.
----------------------------------------------------------------------------------------------------------------------------------------------------------------
Result number 2 with (relative) distance 37.41%: 
 Monitoring: The company retains the right to monitor internet and email usage for security and compliance purposes.
----------------------------------------------------------------------------------------------------------------------------------------------------------------
Result number 3 w

In [ ]:
#################### GENERATION ####################
""" L'approccio standard consiste nell'unire tutti i 'top_k' contesti. Se invece
    volessi unire query-contesto_singolo, allora otterrei 'top_k' risposte, e dovrei utilizzare un altro
    LLM che riassuma. Questo approccio e' verso il re-ranking """

def generate_answer_flan(query, top_contexts):
  concatenated_context = " ".join(top_contexts)
  # usiamo il prompt template tipico di flan-t5
  prompt_template = f"""
  Answer the question based on the context below.\n\n
  Context: {concatenated_context}\n\n
  Question: {query}\n\n
  Answer:"""

  token_info = generator_tokenizer(prompt_template, return_tensors = "pt",
                                   max_length = 512, # lunghezza massima di flan-t5
                                   truncation = True)

  # print("Cosa legge il modello:", generator_tokenizer.decode(token_info['input_ids'][0]))

  outputs = generator.generate(
              **token_info,
              max_new_tokens = 50, # Lunghezza della risposta generata
              num_beams = 4,       # Beam search per qualita' migliore
              early_stopping = True)

  final_answer = generator_tokenizer.decode(outputs[0], skip_special_tokens = True)
  return final_answer

query = "Which is the Drug and Alcohol Policy?"
distances, indices, normalized_distances = retrieve_from_query(query, 5, display = False)
top_contexts = [paragraphs[index] for index in indices]

final_answer = generate_answer_flan(query, top_contexts)

final_answer # to establish clear expectations and guidelines for the responsible use of drugs and alcohol within the organization

'to establish clear expectations and guidelines for the responsible use of drugs and alcohol within the organization'